# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([66])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [10]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    unitary(params[0:6], 1, 2)
    unitary(params[6:12], 3, 4)
    unitary(params[12:18], 5, 6)

    # Second layer
    unitary(params[18:24], 0, 1)
    unitary(params[24:30], 2, 3)
    unitary(params[30:36], 4, 5)
    unitary(params[36:42], 6, 7)

    # Third layer
    unitary(params[42:48], 2, 5)

    # Fourth layer
    unitary(params[48:54], 1, 2)
    unitary(params[54:60], 5, 6)

    # Fifth layer
    unitary(params[60:66], 2, 5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [11]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549]

Example features: [ 3.23006689 -3.38480243  1.9747935   0.71663767 -0.18818383 -0.31244166
  0.66383923  3.90955675]

Expectation value: 0.07567050612275517

 0: ──RY(3.23)────RY(0.224)─────────────────────────────────────────────────────────

# Accuracy test definition

In [12]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [13]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5752701080432172
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6146458583433374
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6451380552220889
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6588235294117647
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.694357743097239
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.692436974789916
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6998799519807923
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.6710684273709484
epoch 71


epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.723889555822329
test_accuracies: [0.6998799519807923, 0.6710684273709484, 0.7056422569027612, 0.7111644657863145, 0.7296518607442977, 0.6991596638655462, 0.7255702280912365, 0.7272509003601441, 0.7272509003601441, 0.7183673469387755, 0.7289315726290516, 0.723889555822329, 0.7241296518607443, 0.7226890756302521, 0.685234093637455, 0.709483793517407, 0.7212484993997599, 0.6972388955582233, 0.7128451380552221, 0.7166866746698679, 0.7178871548619448, 0.678031212484994, 0.702280912364946, 0.7241296518607443, 0.7284513805522208, 0.7145258103241297, 0.7289315726290516, 0.7157262905162065, 0.7186074429771909, 0.723889555822329]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7322929171668667
best accuracy so far!
test_accuracies: [0.6710684273709484, 0.7056422569027612, 0.7111644657863145, 0.7296518607442977, 0.6991596638655462, 0.725570228

epoch 458
epoch 459
epoch 460
accuracy: 0.7270108043217287
test_accuracies: [0.723889555822329, 0.7241296518607443, 0.7226890756302521, 0.685234093637455, 0.709483793517407, 0.7212484993997599, 0.6972388955582233, 0.7128451380552221, 0.7166866746698679, 0.7178871548619448, 0.678031212484994, 0.702280912364946, 0.7241296518607443, 0.7284513805522208, 0.7145258103241297, 0.7289315726290516, 0.7157262905162065, 0.7186074429771909, 0.723889555822329, 0.7322929171668667, 0.7322929171668667, 0.7010804321728692, 0.7226890756302521, 0.721968787515006, 0.714765906362545, 0.707563025210084, 0.7188475390156062, 0.7171668667466986, 0.720048019207683, 0.7270108043217287]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7159663865546219
test_accuracies: [0.7241296518607443, 0.7226890756302521, 0.685234093637455, 0.709483793517407, 0.7212484993997599, 0.6972388955582233, 0.7128451380552221, 0.7166866746698679, 0.7178871548619448, 0.678031

epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.7157262905162065
test_accuracies: [0.7241296518607443, 0.7284513805522208, 0.7145258103241297, 0.7289315726290516, 0.7157262905162065, 0.7186074429771909, 0.723889555822329, 0.7322929171668667, 0.7322929171668667, 0.7010804321728692, 0.7226890756302521, 0.721968787515006, 0.714765906362545, 0.707563025210084, 0.7188475390156062, 0.7171668667466986, 0.720048019207683, 0.7270108043217287, 0.7159663865546219, 0.7130852340936374, 0.7109243697478991, 0.7142857142857143, 0.7327731092436975, 0.7116446578631452, 0.682593037214886, 0.7258103241296519, 0.7207683073229292, 0.7255702280912365, 0.7248499399759905, 0.7157262905162065]
epoch 581
epoch 582
epoch 583
epoch 584
epoch 585
epoch 586
epoch 587
epoch 588
epoch 589
epoch 590
accuracy: 0.7128451380552221
test_accuracies: [0.7284513805522208, 0.7145258103241297, 0.7289315726290516, 0.7157262905162065, 0.7186074429771909, 0.7238895558

epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.7217286914765906
test_accuracies: [0.721968787515006, 0.714765906362545, 0.707563025210084, 0.7188475390156062, 0.7171668667466986, 0.720048019207683, 0.7270108043217287, 0.7159663865546219, 0.7130852340936374, 0.7109243697478991, 0.7142857142857143, 0.7327731092436975, 0.7116446578631452, 0.682593037214886, 0.7258103241296519, 0.7207683073229292, 0.7255702280912365, 0.7248499399759905, 0.7157262905162065, 0.7128451380552221, 0.7289315726290516, 0.7169267707082834, 0.7258103241296519, 0.7174069627851141, 0.7250900360144057, 0.7301320528211285, 0.715486194477791, 0.726530612244898, 0.7234093637454981, 0.7217286914765906]
epoch 691
epoch 692
epoch 693
epoch 694
epoch 695
epoch 696
epoch 697
epoch 698
epoch 699
epoch 700
accuracy: 0.7174069627851141
test_accuracies: [0.714765906362545, 0.707563025210084, 0.7188475390156062, 0.7171668667466986, 0.720048019207683, 0.7270108043217287, 0.7159663865546219, 0.7130852340936374, 0.7109

epoch 800
accuracy: 0.7234093637454981
test_accuracies: [0.7327731092436975, 0.7116446578631452, 0.682593037214886, 0.7258103241296519, 0.7207683073229292, 0.7255702280912365, 0.7248499399759905, 0.7157262905162065, 0.7128451380552221, 0.7289315726290516, 0.7169267707082834, 0.7258103241296519, 0.7174069627851141, 0.7250900360144057, 0.7301320528211285, 0.715486194477791, 0.726530612244898, 0.7234093637454981, 0.7217286914765906, 0.7174069627851141, 0.715486194477791, 0.7090036014405763, 0.7176470588235294, 0.7174069627851141, 0.7198079231692678, 0.7171668667466986, 0.7248499399759905, 0.7186074429771909, 0.7186074429771909, 0.7234093637454981]


In [14]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.23719335  1.74617634  0.10707689  0.17928107  1.34874946 -0.12018666
  0.62182588  0.66387066 -0.53285295  0.82413323  0.95789884  0.92478404
  1.74125082 -0.3929308   1.6495516   1.01487402  2.07425709  0.99880049
 -0.80888763  1.24249981  1.91821918  0.37629386  0.66133746  0.41099934
  0.07349547  0.33793004  1.01191634  0.92605925  0.0453786   0.23132314
  1.1476173   1.65654206  0.51136243  0.42893464  0.72117264  0.24816792
  1.68801166 -0.31013435  0.55476767 -0.0628746  -0.01421495  0.16187284
  0.02268136  0.57953893  0.56913409  0.6177804   0.25176895  1.64450242
  0.81144126 -0.39519673  1.54043331  1.20853128  0.58058546  0.12249713
  1.37793545 -0.20157098 -0.9971355   1.48374415  0.27739942  0.76905057
  0.01654534 -0.06663374  0.48997391  0.29734101  0.11683869 -0.39462305]


# Testing

In [15]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7307619047619047


In [16]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 8.576428413391113
training time: 8293.168436527252
test time: 198.14931082725525
total time: 8499.894175767899
